# TP Métaheuristiques

## 1ère étape

### Lecture des jeux de données

Remarque : Les arcs enregistrés dans les fichiers ne sont pas forcément dans le bon sens.
Exemple : 4 2 <=> 2 4

In [16]:
# Objets
class Arbre:
    """ Informations d'un noeud à évacuer
    - Liste des noeuds evac
    - Id du noeud sécurisé
    """
    def __init__(self):
        self.listeNoeudsE = []
    def sur(self, c):
        self.sur = c[1]
    def ajoutNoeudE(self, c):
        nouveauNoeudE = NoeudAEvacuer(c)
        self.listeNoeudsE.append(nouveauNoeudE)
        
class NoeudAEvacuer:
    # ID/Population
    # Arc de fuite
    def __init__(self, c):
        self.id = c[0]
        self.pop = c[1]
        self.tauxE = c[2]
        self.cheminNoeuds = c[3:]
    def defArcSuivant(self, arc):
        self.arcSuivant = arc
    
class Arc: 
    # Arc suivant(arc)/Durée(int)/Capa(int)
    def __init__(self, c):
        self.arcSuivant = c[0]
        self.pop = c[1]
        self.tauxE = c[2]
        
class Graphe: 
    # Liste d'arcs 
    def __init__(self):
        self.listeArcs = []
    def ajoutArc(self, c):
        nouveauArc = Arc(c)
        self.listeArcs.append(nouveauArc)
        
# -------------------------------

def lecture(nomFichier): 
    with open("Instances/"+nomFichier, "r") as fichier :
        print ("Name of the file: " + fichier.name)

        # Liste des lignes du fichier
        lignes = fichier.readlines()
        
        arbre = Arbre()
        graphe = Graphe()
        
        # Traitement de l'instance
        val = 1
        for ligne in lignes: 
            c = ligne.split()
            if c[0] == 'c': 
                val = 2
            else:
                if val == 1:
                    traitementArbre(c)
                elif val == 2:
                    traitementGraphe(c)
                else :
                    print("Erreur")

    fichier.close()

def traitementArbre(c):
    if len(c) != 2:
        arbre.ajoutNoeudE(c)
    else:
        arbre.sur(c)
        
def traitementGraphe(c):
    if len(c) != 2: 
        graphe.ajoutArc(c)
        


In [ ]:
lecture(exemple_cours.txt)